In [5]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from itertools import product
from sklearn.ensemble import VotingClassifier

In [2]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('F:/notebook_working/kaggle_compe/train.csv').fillna('No data')
test = pd.read_csv('F:/notebook_working/kaggle_compe/test.csv').fillna('No data')

#train.id = train.id.astype('str')
#train.comment_text = train.comment_text.astype('str')

#test.id = test.id.astype('str')
#test.comment_text = test.comment_text.astype('str')

train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])



In [3]:
print train_text.shape
print test_text.shape
#print train.info()
#print test.info()
print all_text.shape
#print all_text

(159571,)
(153164,)
(312735,)


In [3]:

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    encoding='utf-8',
    lowercase=True,
    min_df=0.00001,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    #use_idf=1, smooth_idf=1,
    max_features=None)

word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

msg = "There are {} tokens in Comment_text if we use word"
print(msg.format(len(word_vectorizer.get_feature_names())))


There are 62311 tokens in Comment_text if we use word


In [ ]:
#dont run
char_vectorizer = TfidfVectorizer(
    #norm=None,
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(1,2), 
    #non_negative=True,
    #min_df = 0.00009)
    max_features=10000)
    
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

msg = "There are {} tokens in Comment_text if we use char"
print(msg.format(len(char_vectorizer.get_feature_names())))


In [29]:
#word_vectorizer.get_feature_names()[:100]

In [32]:
#col = [i for i in word_vectorizer.get_feature_names()]
#temp = pd.DataFrame(train_word_features.todense(), columns=col)
#temp


In [9]:
#train_features = hstack([train_char_features, train_word_features])
#test_features = hstack([test_char_features, test_word_features])
#X_train, X_valid, y_train, y_valid = train_test_split(train_features, train_target, test_size=0.3, random_state=42)


scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    
    clf1 = LogisticRegression(solver='sag')
    #clf2 = SVC(kernel='rbf', probability=True)
    clf2 = MultinomialNB()
    
    eclf = VotingClassifier(estimators = [('lr', clf1), ('mnb', clf2)], voting='hard')

    for clf, label in zip([clf1, clf2, eclf], ['Logistic Regression', 'Naive',  'Ensemble']):
        cv_score = np.mean(cross_val_score(clf, train_word_features, train_target, cv=3, scoring='accuracy'))
        #scores.append(cv_score)
        print('CV score for class {} in classifier {} is {}'.format(class_name, label, cv_score))

    #classifier.fit(train_word_features, train_target)
    #submission[class_name] = classifier.predict_proba(test_word_features)[:, 1]

#print('Total CV score is {}'.format(np.mean(scores)))

#submission.to_csv('F:\submission.csv', index=False)

CV score for class toxic in classifier Logistic Regression is 0.954233537219
CV score for class toxic in classifier Naive is 0.931397309383


F:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
F:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
F:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


CV score for class toxic in classifier Ensemble is 0.930049947395
CV score for class severe_toxic in classifier Logistic Regression is 0.990649931143
CV score for class severe_toxic in classifier Naive is 0.989991916135


F:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
F:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
F:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


CV score for class severe_toxic in classifier Ensemble is 0.990004449584
CV score for class obscene in classifier Logistic Regression is 0.976336553479
CV score for class obscene in classifier Naive is 0.958501225847


...

F:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
F:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
F:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


CV score for class insult in classifier Ensemble is 0.956383053458
CV score for class identity_hate in classifier Logistic Regression is 0.991828089809
CV score for class identity_hate in classifier Naive is 0.99118887521


F:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
F:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


CV score for class identity_hate in classifier Ensemble is 0.991195142052


F:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [6]:
#train_features = hstack([train_char_features, train_word_features])
#test_features = hstack([test_char_features, test_word_features])
#X_train, X_valid, y_train, y_valid = train_test_split(train_features, train_target, test_size=0.3, random_state=42)


scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    classifier = MultinomialNB()

    cv_score = np.mean(cross_val_score(classifier, train_word_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_word_features, train_target)
    #submission[class_name] = classifier.predict_proba(test_word_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

#submission.to_csv('F:\submission.csv', index=False)

CV score for class toxic is 0.912236525414
CV score for class severe_toxic is 0.897197060423
CV score for class obscene is 0.909142642501
CV score for class threat is 0.798306584116
CV score for class insult is 0.902539585638
CV score for class identity_hate is 0.84079716009
Total CV score is 0.876703259697


In [ ]:
eclf = VotingClassifier(estimators=[('dt', clf1), ('knn', clf2), ('svc', clf3)], voting='soft', weights=[2,1,2])
eclf.fit()


In [9]:
submission[class_name] = classifier.predict_proba(test_features)[:, 1]
submission.to_csv('submission.csv', index=False)